# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Машинсон Всеволод Валерьевич

Группа: DS12

In [781]:
import osmapi as osm
from PIL import Image
import requests
from io import BytesIO
import geopandas as gpd
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.api import Api
from shapely import geometry
from shapely.geometry import shape, Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import cascaded_union, transform
import folium
import random
from folium.plugins import TimestampedGeoJson
from datetime import datetime, timedelta
import numpy as np
import math
import rtree.index
from pyproj import Transformer, Proj, transform
from openrouteservice import client

def distance_haversine(lat1, lon1, lat2, lon2):
    R = 6371 # km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    a = math.sin(dLat/2.0) * math.sin(dLat/2.0) + math.sin(dLon/2.0) * math.sin(dLon/2.0) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    
    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1)*math.cos(lat2)*math.cos(dLon)
    brng = math.atan2(y,x)
    return d, (brng/(2*math.pi))*360

Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [637]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
afghanistan_gdf = world.loc[world['name'].str.contains('Afghanistan')] # get Singapore row
afghanistan_gdf.crs = "EPSG:4326"
afghanistan_geometry = afghanistan_gdf.geometry.ravel()[0]
afghanistan_center = (afghanistan_geometry.centroid.y, afghanistan_geometry.centroid.x)

In [638]:
m = folium.Map(afghanistan_center, zoom_start=2)
folium.GeoJson(afghanistan_gdf).add_to(m)
folium.Marker(
    location=afghanistan_center,
    tooltip='Afghanistan Center',
    icon=folium.Icon(color='green')
).add_to(m)

def point_distance(tuple_point1, tuple_point2):
    return Point(tuple_point1[1], tuple_point1[0]).distance(Point(tuple_point2[1], tuple_point2[0]))

def random_search(seed = 112):
    MIN_LAT, MAX_LAT, MIN_LON, MAX_LON = -90, 90, -180, 180

    random.seed(seed)
    startPoint = (random.randrange(MIN_LAT, MAX_LAT + 1), random.randrange(MIN_LON, MAX_LON + 1))

    currentPoint = startPoint                                                       
    currentDistance = point_distance(startPoint, afghanistan_center)
    
    i = 0   
    points = [startPoint]
    while not afghanistan_geometry.contains(Point(currentPoint[1], currentPoint[0])):
        add = 5
        down, up, left, right = (currentPoint[0] - add, currentPoint[1]), (currentPoint[0] + add, currentPoint[1]), (currentPoint[0], currentPoint[1] - add), (currentPoint[0], currentPoint[1] + add)
        wasSet = False
        while (True):
            for point in [down, up, left, right]:
                dist = point_distance(point, afghanistan_center)
                if dist < currentDistance:
                    currentDistance = dist
                    currentPoint =  (random.randrange(point[0] - 2, point[0] + 2), random.randrange(point[1] - 2, point[1] + 2)) 
                    wasSet = True
            if wasSet or afghanistan_geometry.contains(Point(currentPoint[1], currentPoint[0])):
                break

            add /= 2
            down, up, left, right = (currentPoint[0] - add, currentPoint[1]), (currentPoint[0] + add, currentPoint[1]), (currentPoint[0], currentPoint[1] - add), (currentPoint[0], currentPoint[1] + add)
        i += 1
        points.append(currentPoint)
    return points
points = random_search(5)

def create_geojson_features(points):
    features = []
    
    for i, p in enumerate(points):
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[p[1], p[0]]
            },
            'properties': {
                'time': (datetime.today() + i * timedelta(hours=0, minutes=1)).strftime("%Y-%m-%d %H:%M:%S"),
                'style': {'color' : ''},
                'icon': '100',
                'iconstyle':{
                    'fillColor': '#00FF00',
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 10000
                }
            }
        }
        features.append(feature)
    return features

json = create_geojson_features(points)
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': json},
    period='PT1M',
    duration='PT5M',
    auto_play=True,
    loop=False,
    loop_button=False
).add_to(m)

m

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

In [ ]:
class Country:
    def __init__(self, osm_pytools_element, areaId):
        self.osm_pytools_element = osm_pytools_element
        self.areaId = areaId
        
    def __str__(self):
        return f'{self.osm_pytools_element} {self.en_name()}: {self.areaId}'
    
    def en_name(self):
        return self.osm_pytools_element.tags()["name:en"]
    
    def name(self):
        return self.osm_pytools_element.tags()["name"]


    
def get_all_countries_with_areaid_ends_to_zero():
    result = Overpass().query('relation["admin_level"="2"][boundary=administrative];out;')
    countries = []

    for el in result.elements():
        name = el.tags()['name']
        areaId = nominatim.query(name).areaId()
        if areaId is not None and areaId % 10 == 0:
            countries.append(Country(el, areaId))
    
    return countries


def count_of_residential(country):
    overpass = Overpass()
    query = overpassQueryBuilder(area=country.areaId, elementType=['node', 'relation'], selector=['"building:use"="residential"'], out='count')
    result = overpass.query(query, timeout=180)
    return result


def count_of_pharmacy_in_capital(country):
    def fix(bb):
        x = list(bb)
        x[1], x[2] = x[2], x[1]
        return x

    overpass = Overpass()
    query = overpassQueryBuilder(area=country.areaId, elementType=['node', 'relation'], selector=['"capital"="yes"'], out='body')
    capitalElements = overpass.query(query, timeout=180).elements()
    if len(capitalElements) == 0:
        return -1
    else:
        capitalElement = capitalElements[0]
    
    nom_result = nominatim.query(capitalElement.tags()["name"]).toJSON()
    for r in nom_result:
        if r['osm_id'] == capitalElement.id():
            query = overpassQueryBuilder(bbox=fix(r['boundingbox']), elementType=['node'], selector='"amenity"="pharmacy"', out='body')
            result = overpass.query(query, timeout=180)
            return len(result.nodes())
        
    return -1


def railway_count_divided_by_bus_stops_count(country):
    bus_stops_query = overpassQueryBuilder(area=country.areaId, elementType=['node', 'relation'], selector=['"public_transport"="stop_position"', '"bus"="yes"'], out='body')
    bus_stops = Overpass().query(bus_stops_query, timeout=180).elements()
    railway_stops_query = overpassQueryBuilder(area=country.areaId, elementType=['node', 'relation'], selector=['"railway"="stop"'], out='body')
    railway_stops = Overpass().query(railway_stops_query, timeout=180).elements()
#     print(f'{country.en_name()}: bus_stops: {len(bus_stops)}  railway_stops: {len(railway_stops)}')
    if len(bus_stops) == 0:
        return -1
    
    return len(railway_stops) / len(bus_stops)

    
    
#Возвращает страну с максимальным количеством обьектов жилой недвижимости
def country_with_max_residential(countries):
    residential_counts = [count_of_residential(c).countElements() for c in countries]
    return countries[np.argmax(residential_counts)]


#Возвращает страну с максимальным количеством аптек в столице
def country_with_max_pharmacy_in_capital(countries):
    pharmacy_in_capital_counts = [count_of_pharmacy_in_capital(c) for c in countries]
    return countries[np.argmax(pharmacy_in_capital_counts)]


#Возвращает страну с максимальным отношением числа жд станций к автобусным остановкам
def country_with_max_railways_to_busstops_relation(countries):
    railway_to_busstops_relations = [railway_count_divided_by_bus_stops_count(c) for c in countries]
    return countries[np.argmax(railway_to_busstops_relations)]


def get_elements(country, amenity):
    overpass = Overpass()
    query = overpassQueryBuilder(area=country.areaId, elementType=['node', 'relation'], selector=[f'"amenity"="{amenity}"'], out='body', includeGeometry=True)
    result = overpass.query(query, timeout=180)
    return result.elements()

def get_atms(country):
    return get_elements(country, 'atm')

def get_schools(country):
    return get_elements(country, 'school')

def get_сinemas(country):
    return get_elements(country, 'cinema')

#-----------------------------------------Rtree---------------------------------------
def sum_of_distances_to_5_nearest_items(target_lat, target_lon, elements):
    nearest = find_nearest_items(target_lat, target_lon, 5, elements)
    result = 0
    for item in nearest:
        bounds = bounds_function(item.object)
        lat, lon = bounds[0], bounds[1]
        result += distance_haversine(lat, lon, target_lat, target_lon)[0]
    return result


def bounds_function(element):
    coordinates = element.geometry()["coordinates"]
    if element.type() == 'node':
        lat, lon = coordinates[1], coordinates[0]
        return (lat, lon, lat, lon) 
    elif element.type() == 'relation':
        flat_list = [item for sublist in coordinates for item in sublist]
        arr = np.asarray(flat_list)
        longitudes, latitudes = arr[:, 0], arr[:, 1]
        min_lat, min_lon, max_lat, max_lon = latitudes.min(), longitudes.min(), latitudes.max(), longitudes.max()
        return (min_lat, min_lon, max_lat, max_lon)
    else:
        assert(false)
    

def find_nearest_items(lat, lon, k, elements):
    idx = rtree.index.Rtree()
    assert(idx.interleaved)
    for i, element in enumerate(elements):
        bounds = bounds_function(element)
        idx.insert(i, bounds, obj = element)
    
    return idx.nearest((lat, lon, lat, lon), k, objects=True)

#-----------------------------------------Solutions---------------------------------------
def solve_task_2_1():
    countries = get_all_countries_with_areaid_ends_to_zero()
    country = country_with_max_residential(countries)
    result = sum_of_distances_to_5_nearest_items(afghanistan_center[0], afghanistan_center[1], get_atms(country))
    print(f'Сумма расстояний от точки до 5 ближайших * банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости: {result} км')
    
    
def solve_task_2_2():
    countries = get_all_countries_with_areaid_ends_to_zero()
    country = country_with_max_pharmacy_in_capital(countries)
    result = sum_of_distances_to_5_nearest_items(afghanistan_center[0], afghanistan_center[1], get_schools(country))
    print(f'Сумма расстояний от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице: {result} км')
    

def solve_task_2_3():
    countries = get_all_countries_with_areaid_ends_to_zero()
    country = country_with_max_railways_to_busstops_relation(countries)
    result = sum_of_distances_to_5_nearest_items(afghanistan_center[0], afghanistan_center[1], get_сinemas(country))
    print(f'Сумма расстояний от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части : {result} км')

In [224]:
solve_task_2_1()

Сумма расстояний от точки до 5 ближайших * банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости: 57045.17318902758 км


In [640]:
solve_task_2_2()

Сумма расстояний от точки до Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице: 18871.489575627777 км


In [660]:
solve_task_2_3()

Сумма расстояний от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части : 18302.97479946772 км


## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [844]:
api_key = '5b3ce3597851110001cf6248710f068cbe5549adaf09a2bbd1d67af6' 
clnt = client.Client(key=api_key)
def CreateBufferPolygon(point_in, resolution=10, radius=100):    
    crs1 = "epsg:4326"
    crs2 = "epsg:3857"
    transformer = Transformer.from_crs(crs1, crs2, always_xy=True)
    transformer_back = Transformer.from_crs(crs2, crs1, always_xy=True)
   
    point_in_proj = transformer.transform(*point_in) # unpack list to arguments
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution) # 10 m buffer
    
    # Iterate over all points in buffer and build polygon
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(transformer_back.transform(*point)) # Transform back to WGS84

    return poly_wgs


# Function to request directions with avoided_polygon feature
def CreateRoute(coordinates, avoided_point_list, n=0):
    route_request = {'coordinates': coordinates, 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                    'instructions': False,
                     'options': {'avoid_polygons': geometry.mapping(MultiPolygon(avoided_point_list))}} 
    route_directions = clnt.directions(**route_request)
    
    return route_directions


# Function to create buffer around requested route
def CreateBuffer(route_directions): 
    line_tup = []
    for line in route_directions['features'][0]['geometry']['coordinates']:
        tup_format = tuple(line)
        line_tup.append(tup_format)  

    new_linestring = LineString(line_tup)
    dilated_route = new_linestring.buffer(0.0002)
        
    return dilated_route
    
    
def style_function(color): # To style data
    return lambda feature: dict(color=color)


class Constants:
    start_point = [40.777119, -73.963741]
    finish_point = [40.703214, -73.989597]
    coordinates_lon_lat_order = [list(reversed(Constants.start_point)), list(reversed(Constants.finish_point))]
    

def visualize_schools(follium_map):
    gpdf = gpd.read_file('Schools.csv')
    gpdf.crs = "EPSG:4326"
    school_buffer_geometries = []
   
    for point in list(gpdf[:600].the_geom):
        lon, lat = map(float, point.replace("POINT","").replace("(","").replace(")","").split())
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,
            color="#FF0000"
        ).add_to(follium_map)

        #lon lat order because we set always xy = true
        buffer = CreateBufferPolygon([lon, lat], resolution = 10, radius = 100)
        school_buffer_geometries.append(Polygon(buffer))

    union_poly = mapping(cascaded_union(school_buffer_geometries))                     
    folium.features.GeoJson(data=union_poly,
                        name='Flood affected areas',
                        style_function=style_function('#00FF00')).add_to(follium_map) 
    return school_buffer_geometries 

def visualize_route(follium_map, school_buffer_geometries):
    # Visualize start and destination point on map
    folium.Marker(
        location=Constants.start_point,
        tooltip='Start point',
        icon=folium.Icon(color='green')
    ).add_to(follium_map)
    
    folium.Marker(
        location=Constants.finish_point,
        tooltip='Finish point',
        icon=folium.Icon(color='black')
    ).add_to(follium_map)


    avoided_point_list = []
    route_directions = CreateRoute(Constants.coordinates_lon_lat_order, avoided_point_list)

    folium.features.GeoJson(
        data=route_directions,
        name='Regular Route',
        style_function=style_function('#ff5050'),
        overlay=True
    ).add_to(follium_map)
    
    print('Generated regular route.')

    dilated_route = CreateBuffer(route_directions) # Create buffer around route

    try:
        route_directions = CreateRoute(Constants.coordinates_lon_lat_order, school_buffer_geometries, len(school_buffer_geometries))
        folium.features.GeoJson(
            data=route_directions,
            name='Alternative Route',
            style_function=style_function('#0000FF'),
            overlay=True
        ).add_to(follium_map)
        print('Generated alternative route, which avoids affected areas.')
    except Exception: 
        print('Sorry, there is no route available between the requested destination because of too many blocked streets.')

    follium_map.add_child(folium.map.LayerControl())
    follium_map
    



folium_map = folium.Map(location=Constants.start_point, zoom_start=14) # Create map 
school_buffer_geometries = visualize_schools(folium_map)
visualize_route(folium_map, school_buffer_geometries)
folium_map

Generated regular route.
Generated alternative route, which avoids affected areas.
